## Implementation

In [1]:
mutable struct fourmi
    pos ::Array{Int,1} 
    food ::Int # 1= has food, 0 = no food, -1 = has found an empty source
    dir ::Array{Int,1} 
    function fourmi(pos=[81,81],food=0,dir=rand([[1,0],[-1,0],[0,1],[0,-1]]))
        new(pos,food,dir)
    end
end

In [2]:
function generate()
    fm=ones(Int,161,161) #1 because 0 can lead to cases where only one way is used
    fn=ones(Int,161,161)
    ff=Array{fourmi,1}()
    source=-1*ones(Int,161,161)
    ff,source,fm,fn
end

generate (generic function with 1 method)

In [3]:
function next_pos(f,fer)
    nextpos=[f.pos+f.dir,f.pos+f.dir+[0 1;-1 0]*f.dir,f.pos+f.dir+[0 -1;1 0]*f.dir]# use of the rotation matrix to find the coordinates of the possible next positions
    tochoose=Array{Int,1}() # this is a vector representing the probabilities, each possible next position is represented f (n or m) times
    for (i,p) in enumerate(nextpos)
        append!(tochoose,i*ones(Int,1,fer[p...]))
    end
    next=nextpos[rand(tochoose)]
end

next_pos (generic function with 1 method)

In [4]:
function turn(ff,source,fm,fn)
    source2=deepcopy(source)
    fm2=deepcopy(fm)
    fn2=deepcopy(fn)

    for (i,f) in enumerate(ff)
        ## is it too far or too close to the nest ?
        if sum(abs.(f.pos - [81,81]))<=2 && f.food ≠ 0 # too close and has seen a source --> it is coming back and has to turn around
            f.food=0
            f.dir=-f.dir
            
        elseif minimum(f.pos)==1 ||maximum(f.pos)==161 #too far --> lost
            deleteat!(ff,i)
            continue
        end
        ## has it found a source ?
        if source[f.pos...] > 0 && f.food==0 #found a source
            f.food= 1
            f.dir=-f.dir
            source2[f.pos...]-=1
        elseif source[f.pos...] ==0 && f.food==0 #found a source but this is empty
            f.food=-1
            f.dir=-f.dir
        end
        ## what to do next ?
        if f.food==1 #has to go to the nest
            fn2[f.pos...]+=1
            f.pos=next_pos(f,fm)
        elseif f.food==0 #has to go search for food
            fm2[f.pos...]+=1
            f.pos=next_pos(f,fn)
        else #is coming back and erasing fn
            if fn2[f.pos...]>1
                fn2[f.pos...]-=1
            end
            f.pos=next_pos(f,fm)
        end

    end
    ff,source2,fm2,fn2
end

turn (generic function with 1 method)

## Tests

# 1

In [ ]:
ff,source,fm,fn=generate()
source[70,:]=3*ones(1,161)
push!(ff,fourmi([81,81],0,[-1,0]))

for t in 1:20
    ff,source,fm,fn=deepcopy(turn(ff,source,fm,fn))
    println(ff[1].pos)

end

# 2

In [6]:
ff,source,fm,fn=generate()
source[70,:]=3*ones(1,161)
push!(ff,fourmi([81,81],0,[-1,0]))

for t in 0:25
    if mod(t,2)==0
        push!(ff,fourmi([81,81],0,[-1,0]))
    end
    ff,source,fm,fn=deepcopy(turn(ff,source,fm,fn))
    println(ff[1].pos)
    f=ff[1]
    println(fn[(f.pos+f.dir)...]," ",fn[(f.pos+f.dir+[0 1;-1 0]*f.dir)...]," ",fn[(f.pos+f.dir+[0 -1;1 0]*f.dir)...])
    println(fm[(f.pos+f.dir)...]," ",fm[(f.pos+f.dir+[0 1;-1 0]*f.dir)...]," ",fm[(f.pos+f.dir+[0 -1;1 0]*f.dir)...])
end

[80, 82]
1 1 1
1 1 1
[79, 83]
1 1 1
1 1 1
[78, 82]
1 1 1
1 1 1
[77, 83]
1 1 1
1 1 1
[76, 84]
1 1 1
1 1 1
[75, 83]
1 1 1
1 1 1
[74, 83]
1 1 1
1 1 1
[73, 82]
1 1 1
1 1 1
[72, 81]
1 1 1
1 1 1
[71, 82]
1 1 1
1 1 1
[70, 83]
1 1 1
1 1 1
[71, 82]
1 1 1
2 2 1
[72, 83]
1 1 1
1 2 1
[73, 82]
1 1 1
1 2 2
[74, 83]
1 1 1
2 1 1
[75, 83]
1 1 1
1 1 2
[76, 84]
1 1 1
1 2 1
[77, 84]
1 1 1
1 1 1
[78, 84]
1 1 1
1 2 1
[79, 85]
1 1 1
1 1 1
[80, 84]
1 1 1
1 1 1
[81, 84]
1 1 1
1 1 1
[82, 85]
1 1 1
1 1 1
[83, 84]
1 1 1
1 1 1
[84, 83]
1 1 1
1 1 1
[85, 83]
1 1 1
1 1 1


## 3

In [ ]:
ff,source,fm,fn=generate()
source[70,:]=3*ones(1,161)

for t in 0:50
    if mod(t,2)==0
        push!(ff,fourmi([81,81],0,[-1,0]))
    end
    ff,source,fm,fn=deepcopy(turn(ff,source,fm,fn))
    for (i,f) in enumerate(ff)
        if f.food == -1
            println(i," ",f.pos)
        end
    end
end

## total

In [5]:
using Plots

In [6]:
global mycmap = ColorGradient([RGBA(255/255,255/255,255/255),
    RGBA(0/255,0/255,0/255),
    RGBA(255/255,100/255,100/255),
    RGBA(255/255,0/255,0/255),
    RGBA(0/255,255/255,0/255),
    RGBA(0/255,0/255,255/255)]);

function visualize2dim(ff,source, dim,n)
    nx ,ny =size(source)
    out=Array{Float64,2}(undef,nx,ny)
    for i in 1:nx
        for j in 1:ny
            if source[i,j]==-1
               out[i,j]=fn[i,j]/(n/16)
            elseif source[i,j]>0
                out[i,j]=3
            else
                out[i,j]=2
            end
        end
    end
    #
    for f in ff
        out[f.pos...]=4
    end
    out[81,81]=5
    heatmap(out,yflip=true,color=mycmap,size=(600,600), title="Ant simulation")
end

visualize2dim (generic function with 1 method)

In [ ]:
ff,source,fm,fn=generate()
n=100
for _ in 1:n
    source[rand(1:161),rand(1:161)]=rand(1:20)
end
global ff = deepcopy(ff)
global source= source
global fm  = fm
global fn=fn

anim = @animate for t in 0:10000
    if mod(t,2)==0
        push!(ff,fourmi())
    end
    ff,source,fm,fn=deepcopy(turn(ff,source,fm,fn))
    global ff = ff
    global source= source
    global fm  = fm
    global fn=fn
    visualize2dim(ff,source,3,1000)
end
gif(anim, "anim_fps30.gif", fps = 30)

In [83]:
[0 -1;1 0]^4 *[1,1]

2-element Array{Int64,1}:
 1
 1